In [107]:
import re

class Node:
    def __init__(self, element_type, element_value):
        self.set_node(element_type, element_value)
        self.children = []

    def add_child(self, x_node):
        self.children.append(x_node)

    def set_child_nodes(self, children):
        self.children = children

    def get_child_nodes(self):
        return self.children

    def set_node(self, element_type, element_value):
        self.element_type = element_type
        self.element_value = element_value

    def get_element_type(self):
        return self.element_type

    def get_element_value(self):
        return self.element_value

    def get_text(self):
        return "[" + self.get_element_type() + "] " + self.get_element_value()

    def __str__(self, level=0):
        text = "--" * level + self.get_text() + "\n"
        for child_node in self.children:
            text += child_node.__str__(level + 1)
        return text


OPS = ['AND', 'OR', 'IMPLIES', 'NOT','IF']
QUANTS = ['FORALL', 'EXISTS']

def parse_tree(args):
    _stack = []

    _stack_element = None
    for index in range(len(args)):
        current_index = index

        current_element = args[current_index]
        current_symbol_type = current_element[0]
        current_symbol_value = current_element[1]

        if current_symbol_type == "open_bracket" and current_symbol_value == "(":
            continue
        elif current_symbol_type == "close_bracket" and current_symbol_value == ")":
            _picked_child_nodes = []
            while True:
                _parent_node = _stack.pop()

                _symbol_type = _parent_node.get_element_type()
                _symbol_value = _parent_node.get_element_value()

                if _symbol_type == "op" or _symbol_type == "quant" or _symbol_type == "function" or _symbol_type == "predicate":
                    _child_nodes = _parent_node.get_child_nodes()
                    _no_child_nodes = len(_child_nodes)

                    if _no_child_nodes == 0:
                        break

                _picked_child_nodes.append(_parent_node)

            _parent_node.set_child_nodes(_picked_child_nodes)

            _stack.append(_parent_node)

        else:
            _node = Node(current_symbol_type, current_symbol_value)
            _stack.append(_node)

    assert (len(_stack) == 1)

    return _stack.pop()


def parse(F):
    characters = F
    # breaking the input to argument types
    # argument types: open and close bracket, operator and symbol
    args = []
    regex = r'''\(|\)|\[|\]|\-?\d+\.\d+|\-?\d+|[^,(^)\s]+'''

    # sanitizing the input
    characters = characters.replace("\t", " ")
    characters = characters.replace("\n", " ")
    characters = characters.replace("\r", " ")
    characters = characters.lstrip(" ")
    characters = characters.rstrip(" ")

    prev_arg = next_arg = None
    lines = []
    arg_list = re.findall(regex, characters)
    for i in range(len(arg_list)):
        arg = arg_list[i]
        if (i - 1 >= 0):
            prev_arg = arg_list[i - 1]
        if (i + 1 < len(arg_list)):
            next_arg = arg_list[i + 1]

        if (arg == "("):
            arg_name = "open_bracket"
        elif (arg == ")"):
            arg_name = "close_bracket"
        elif prev_arg == "(":
            if (arg in OPS):
                arg_name = "op"
            elif (arg in QUANTS):
                arg_name = "quant"
            else:
                arg_name = "function"
        elif (prev_arg in QUANTS):
            arg_name = "variable"
        elif arg.isalnum():
            arg_name = "symbol"

        arg_tuple = (arg_name, arg)
        args.append(arg_tuple)

    return parse_tree(args)


def eliminate_implication(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() == "IMPLIES":
        left_child, right_child = formula.get_child_nodes()
        # Replace implication with its equivalent form: (NOT A) OR B
        new_formula = Node("op", "OR")
        not_left_child = Node("op", "NOT")
        not_left_child.add_child(left_child)
        new_formula.add_child(not_left_child)
        new_formula.add_child(right_child)
        return new_formula
    else:
        for child_node in formula.get_child_nodes():
            eliminate_implication(child_node)
    return formula

def move_negation_inward(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() == "NOT":
        child_node = formula.get_child_nodes()[0]
        if child_node.get_element_type() == "op":
            operator = child_node.get_element_value()
            if operator == "AND":
                # Apply Demorgan's Law: NOT (A AND B) = (NOT A) OR (NOT B)
                formula.set_node("op", "OR")
                not_left_child = Node("op", "NOT")
                not_left_child.add_child(child_node.get_child_nodes()[0])
                not_right_child = Node("op", "NOT")
                not_right_child.add_child(child_node.get_child_nodes()[1])
                formula.set_child_nodes([not_left_child, not_right_child])
            elif operator == "OR":
                # Apply Demorgan's Law: NOT (A OR B) = (NOT A) AND (NOT B)
                formula.set_node("op", "AND")
                not_left_child = Node("op", "NOT")
                not_left_child.add_child(child_node.get_child_nodes()[0])
                not_right_child = Node("op", "NOT")
                not_right_child.add_child(child_node.get_child_nodes()[1])
                formula.set_child_nodes([not_left_child, not_right_child])
            elif operator == "NOT":
                # Double negation elimination: NOT (NOT A) = A
                return child_node.get_child_nodes()[0]
        else:
            # Negate the symbol
            symbol_node = Node("op", "NOT")
            symbol_node.add_child(child_node)
            return symbol_node
    else:
        for child_node in formula.get_child_nodes():
            move_negation_inward(child_node)
    return formula

def remove_double_not(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() == "NOT":
        child_node = formula.get_child_nodes()[0]
        if child_node.get_element_type() == "op" and child_node.get_element_value() == "NOT":
            # Remove double negation: NOT (NOT A) = A
            return child_node.get_child_nodes()[0]
    else:
        for child_node in formula.get_child_nodes():
            remove_double_not(child_node)
    return formula

def standardize_variable_scope(formula, variable_map=None):
    if variable_map is None:
        variable_map = {}

    if formula.get_element_type() == "quant":
        variable = formula.get_element_value()
        if variable in variable_map:
            # If the variable already exists in the map, rename it
            new_variable = variable + "'"
            variable_map[variable] = new_variable
            formula.set_node("quant", new_variable)
        else:
            variable_map[variable] = variable

    for child_node in formula.get_child_nodes():
        standardize_variable_scope(child_node, variable_map)

    return formula

def prenex_form(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() in QUANTS:
        quantifier_node = formula
        quantifier = quantifier_node.get_element_value()
        matrix_node = quantifier_node.get_child_nodes()[0]
        # Move quantifier to the left
        while matrix_node.get_element_type() == "op" and matrix_node.get_element_value() in OPS:
            next_matrix_node = matrix_node.get_child_nodes()[0]
            quantifier_node.set_child_nodes([next_matrix_node])
            matrix_node.set_child_nodes([quantifier_node])
            matrix_node = next_matrix_node
        # Recursively apply prenex form to the new matrix node
        matrix_node = prenex_form(matrix_node)

        return matrix_node
    else:
        for child_node in formula.get_child_nodes():
            prenex_form(child_node)
        return formula

def skolemization(formula, skolem_functions=None):
    if skolem_functions is None:
        skolem_functions = {}

    if formula.get_element_type() == "op" and formula.get_element_value() == "EXISTS":
        # Apply Skolemization
        quantifier_node = formula
        matrix_node = quantifier_node.get_child_nodes()[0]
        
        # Collect universally quantified variables
        universal_vars = set()
        collect_universal_vars(matrix_node, universal_vars)

        # Generate Skolem function name
        skolem_func_name = f"f_{len(skolem_functions)}"

        # Replace existential quantifier with Skolem function
        skolem_functions[skolem_func_name] = list(universal_vars)
        skolem_func_node = Node("function", skolem_func_name)
        matrix_node = substitute_quantifier_with_skolem_func(matrix_node, skolem_func_node)

        # Recursively apply Skolemization to the new matrix node
        matrix_node = skolemization(matrix_node, skolem_functions)

        return matrix_node
    else:
        for child_node in formula.get_child_nodes():
            skolemization(child_node, skolem_functions)

    return formula

def collect_universal_vars(node, universal_vars):
    if node.get_element_type() == "op" and node.get_element_value() in QUANTS:
        pass
    elif node.get_element_type() == "variable":
        universal_vars.add(node.get_element_value())
    else:
        for child_node in node.get_child_nodes():
            collect_universal_vars(child_node, universal_vars)

def substitute_quantifier_with_skolem_func(node, skolem_func_node):
    if node.get_element_type() == "op" and node.get_element_value() in QUANTS:
        return skolem_func_node
    else:
        new_child_nodes = []
        for child_node in node.get_child_nodes():
            new_child_nodes.append(substitute_quantifier_with_skolem_func(child_node, skolem_func_node))
        node.set_child_nodes(new_child_nodes)
        return node

def eliminate_universal_quantifiers(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() == "FORALL":
        # Remove universal quantifier
        return formula.get_child_nodes()[0]
    else:
        for child_node in formula.get_child_nodes():
            eliminate_universal_quantifiers(child_node)
    return formula


def convert_to_cnf(formula):
    formula = eliminate_implication(formula)
    formula = move_negation_inward(formula)
    formula = remove_double_not(formula)
    formula = distribute_disjunction_over_conjunction(formula)
    return formula

def distribute_disjunction_over_conjunction(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() == "OR":
        # Apply distribution of disjunction over conjunction
        left_child, right_child = formula.get_child_nodes()
        if left_child.get_element_type() == "op" and left_child.get_element_value() == "AND":
            # (A AND B) OR C = (A OR C) AND (B OR C)
            new_left_child = Node("op", "OR")
            new_right_child = Node("op", "OR")
            for child_node in left_child.get_child_nodes():
                new_left_child.add_child(child_node)
                new_right_child.add_child(Node(child_node.get_element_type(), right_child.get_element_value()))
            new_formula = Node("op", "AND")
            new_formula.add_child(new_left_child)
            new_formula.add_child(new_right_child)
            return distribute_disjunction_over_conjunction(new_formula)
        elif right_child.get_element_type() == "op" and right_child.get_element_value() == "AND":
            # C OR (A AND B) = (C OR A) AND (C OR B)
            new_left_child = Node("op", "OR")
            new_right_child = Node("op", "OR")
            for child_node in right_child.get_child_nodes():
                new_left_child.add_child(Node(child_node.get_element_type(), left_child.get_element_value()))
                new_right_child.add_child(child_node)
            new_formula = Node("op", "AND")
            new_formula.add_child(new_left_child)
            new_formula.add_child(new_right_child)
            return distribute_disjunction_over_conjunction(new_formula)
    else:
        for child_node in formula.get_child_nodes():
            distribute_disjunction_over_conjunction(child_node)
    return formula

def turn_conjunctions_into_clauses(formula):
    if formula.get_element_type() == "op" and formula.get_element_value() == "AND":
        # Transform conjunction into a set of clauses
        clauses = []
        for child_node in formula.get_child_nodes():
            clause = []
            collect_literals(child_node, clause)
            clauses.append(clause)
        return clauses
    else:
        return None

def collect_literals(node, clause):
    if node.get_element_type() == "op" and node.get_element_value() == "OR":
        for child_node in node.get_child_nodes():
            if child_node.get_element_type() == "op" and child_node.get_element_value() == "NOT":
                literal = child_node.get_child_nodes()[0]
                clause.append((literal.get_element_value(), False))
            else:
                clause.append((child_node.get_element_value(), True))
    else:
        if node.get_element_type() == "op" and node.get_element_value() == "NOT":
            literal = node.get_child_nodes()[0]
            clause.append((literal.get_element_value(), False))
        else:
            clause.append((node.get_element_value(), True))

def rename_variables_in_clauses(clauses):
    variable_map = {}
    for clause in clauses:
        for literal in clause:
            var_name = literal[0]
            if var_name not in variable_map:
                variable_map[var_name] = f"_{len(variable_map)}"
            literal = (variable_map[var_name], literal[1])
    return clauses


def parse_and_apply_resolution(*F):
    # Parse the input formulas
    parsed_formulas = [parse(formula) for formula in F]

    # Apply resolution procedure steps
    for idx, parsed_formula in enumerate(parsed_formulas):
        parsed_formula = eliminate_implication(parsed_formula)
        parsed_formula = move_negation_inward(parsed_formula)
        parsed_formula = remove_double_not(parsed_formula)
        parsed_formula = standardize_variable_scope(parsed_formula)
        parsed_formula = prenex_form(parsed_formula)
        parsed_formula = skolemization(parsed_formula)
        parsed_formula = eliminate_universal_quantifiers(parsed_formula)
        parsed_formula = convert_to_cnf(parsed_formula)
        parsed_formulas[idx] = parsed_formula

    return parsed_formulas

# Test the function
test_cases = [
    ["(FORALL x (IMPLIES (p x) (q x)))", "(p (f a))", "(NOT (q (f a)))"],
    ["(FORALL x (IMPLIES (p x) (q x)))", "(FORALL x (p x))", "(NOT (FORALL x (q x)))"],
    ["(EXISTS x (AND (p x) (q b)))", "(FORALL x (p x))"],
]

for test_case in test_cases:
    parsed_formulas = parse_and_apply_resolution(*test_case[:-1])
    resolved_formula = parsed_formulas[0]  # Assuming the first formula is the main one
    expected_result = parse(test_case[-1])
    print("Resolved Formula:")
    print(resolved_formula)
    print("Expected Result:")
    print(expected_result)
    print("Test Result:")
    print("Pass" if resolved_formula == expected_result else "Fail")
    print()




Resolved Formula:
[quant] FORALL
--[op] IMPLIES
----[function] q
------[symbol] x
----[function] p
------[symbol] x
--[variable] x

Expected Result:
[op] NOT
--[function] q
----[function] f
------[symbol] a

Test Result:
Fail

Resolved Formula:
[quant] FORALL
--[op] IMPLIES
----[function] q
------[symbol] x
----[function] p
------[symbol] x
--[variable] x

Expected Result:
[op] NOT
--[quant] FORALL
----[function] q
------[symbol] x
----[variable] x

Test Result:
Fail

Resolved Formula:
[quant] EXISTS
--[op] AND
----[function] q
------[symbol] b
----[function] p
------[symbol] x
--[variable] x

Expected Result:
[quant] FORALL
--[function] p
----[symbol] x
--[variable] x

Test Result:
Fail
